In [1]:
#Importing Dependencies
%matplotlib inline
import pandas as pd
import numpy as np
from numpy import random
import tensorflow as tf
import matplotlib.pyplot as plt
import plotly.express as px
import sklearn as skl
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn import preprocessing
from sklearn import utils
from functools import reduce
from sklearn.cluster import KMeans
import psycopg2
from sqlalchemy import create_engine
from config import DB_HOST, DB_USER, DB_PASS, DB_PORT

In [2]:
#Creating engine to access online databse
engine = create_engine(f'postgresql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/us_emissions_db')

In [3]:
#reading merged table and year averages table
merged_table = pd.read_sql_table('gdp_pop_sector_emissions', con=engine)
year_avg = pd.read_sql_table('year_averages', con=engine)
year_avg.columns = ['year', 'avgpop', 'avggdp', 'avgghg']

In [4]:
#displaying merged table and year table
display(merged_table.head())
display(year_avg.head())

,index,Country,State,Year,GDP,Population,Sector,allghg,CH4,CO2,N2O,F-Gas,outcomes
0,0,US,AL,1997,3.724787e+10,4367935,Agriculture,6.889241,3.392917,0.016045,4.028508,1.128290,0
1,1,US,AL,1998,3.936806e+10,4404701,Agriculture,6.954451,3.137611,0.016269,3.751630,1.290703,1
2,2,US,AL,1999,4.151366e+10,4430141,Agriculture,6.560038,3.074048,0.013043,3.880403,1.376542,0
3,3,US,AL,2000,4.258337e+10,4452173,Agriculture,6.298967,2.959365,0.002420,3.600673,1.521127,0
4,4,US,AL,2001,4.334858e+10,4467634,Agriculture,6.374233,2.899730,0.007422,3.399237,1.600098,1


,year,avgpop,avggdp,avgghg
0,1997,5.346018e+06,6.336021e+10,19.723300
1,1998,5.408904e+06,6.529091e+10,19.659627
2,1999,5.471376e+06,6.939920e+10,19.800945
3,2000,5.532596e+06,7.215153e+10,20.297083
4,2001,5.587627e+06,7.477159e+10,19.961042


In [81]:
# Encode State, Sector, and Year into Labels
le = LabelEncoder()
encoded_table = merged_table.copy()
encoded_table['StateEncoded'] = le.fit_transform(encoded_table['State'])
# encoded_table['Year'] = le.fit_transform(encoded_table['Year'])
encoded_table['SectorEncoded'] = le.fit_transform(encoded_table['Sector'])
# encoded_table_no_lucf = encoded_table.loc[encoded_table['Sector'] != 'Land-Use Change and Forestry']
encoded_table = encoded_table.copy()
# encoded_table = encoded_table.loc[encoded_table['allghg'] != 0]
encoded_table

,index,Country,State,Year,GDP,Population,Sector,allghg,CH4,CO2,N2O,F-Gas,outcomes,StateEncoded,SectorEncoded
0,0,US,AL,1997,3.724787e+10,4367935,Agriculture,6.889241,3.392917,0.016045,4.028508,1.128290,0,1,0
1,1,US,AL,1998,3.936806e+10,4404701,Agriculture,6.954451,3.137611,0.016269,3.751630,1.290703,1,1,0
2,2,US,AL,1999,4.151366e+10,4430141,Agriculture,6.560038,3.074048,0.013043,3.880403,1.376542,0,1,0
3,3,US,AL,2000,4.258337e+10,4452173,Agriculture,6.298967,2.959365,0.002420,3.600673,1.521127,0,1,0
4,4,US,AL,2001,4.334858e+10,4467634,Agriculture,6.374233,2.899730,0.007422,3.399237,1.600098,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13459,13459,US,WY,2014,1.827150e+10,583159,Waste,0.449731,0.000000,0.000000,0.000000,0.000000,1,50,11
13460,13460,US,WY,2015,1.651612e+10,586389,Waste,0.456765,0.000000,0.000000,0.000000,0.000000,1,50,11
13461,13461,US,WY,2016,1.516490e+10,585243,Waste,0.463380,0.000000,0.000000,0.000000,0.000000,1,50,11
13462,13462,US,WY,2017,1.597636e+10,579994,Waste,0.469695,0.000000,0.000000,0.000000,0.000000,1,50,11


# Random Forest Regressor Model

In [82]:
#Creating target and features for Random Forest Regressor Model to be used on the Avg Table
# X = year_avg[['avgpop','avggdp']]
# y = year_avg['avgghg']
X = encoded_table[['StateEncoded', 'SectorEncoded','GDP', 'Population', 'allghg']]
y = encoded_table['outcomes']

In [83]:
#Splitting the data
X_train, X_test, y_train, y_test = train_test_split(X, y)
# scaler = StandardScaler().fit(X_train.values)
# X_train_scaled = scaler.transform(X_train.values)
# X_test_scaled = scaler.transform(X_test.values)

In [84]:
#Train and test data
clf = RandomForestRegressor(n_estimators=150).fit(X_train.values, y_train.values)

print(f'Training Score: {clf.score(X_train.values, y_train.values)}')
print(f'Testing Score: {clf.score(X_test.values, y_test.values)}')

Training Score: 0.8726755150828334
Testing Score: 0.07556543279324768


In [85]:
#Create function to predict ghg based on Population & GDP per state
def rfr_model(state, sector):
    model_data = encoded_table.loc[(encoded_table['State']==state) & (encoded_table['Sector']==sector)]
    X = model_data[['StateEncoded','SectorEncoded','GDP', 'Population', 'allghg']]
    y = model_data['outcomes']
#     scaler = StandardScaler().fit(X.values)
#     X_test_scaled = scaler.transform(X.values)
#    print(f'Testing Score: {clf.score(X, y)}')
    return clf.score(X.values, y.values)

In [86]:
print(f"Testing Score: {rfr_model('NJ', 'Transportation')}")

Testing Score: 0.5792459259259259


In [52]:
# Create table of State-Sector-Score
state_list = []
sector_list = []
score_list = []
for row in encoded_table.itertuples():
    state = row.State
    sector = row.Sector
    score = rfr_model(state, sector)
    state_list.append(state)
    sector_list.append(sector)
    score_list.append(score)

score_df = pd.DataFrame({'state': state_list, 'sector': sector_list, 'score': score_list}, 
                        columns = ['state', 'sector', 'score'])

In [53]:
# Reduce table to unique State-Sector keys and remove 1.0 scores
score_df.drop_duplicates(inplace=True, ignore_index=True)
score_df = score_df.loc[score_df['score'] != 1]

score_df

,state,sector,score
0,AL,Agriculture,0.605255
1,AL,Bunker Fuels,0.713661
2,AL,Commercial,0.559991
3,AL,Electricity/Heat,0.592882
4,AL,Energy,0.469341
...,...,...,...
560,WY,Industry,0.663492
561,WY,Land-Use Change and Forestry,0.709252
562,WY,Residential,0.654431
563,WY,Transportation,0.509884


In [54]:
# Box Plot of scores by sector
fig = px.box(score_df, y="score", x="sector", points='all')
fig.show()

In [55]:
# Box Plot of scores by state
fig = px.box(score_df, y="score", x="state")
fig.show()

In [56]:
score_df['score'].max()

0.9150891666666666

In [57]:
score_df['score'].min()

0.040146666666666664